# 前処理 (人間行動認識 p.27~)
観測した生データに対して処理を行い，機械学習に適した形に変換する

### 一般的な手順
1. クレンジング
1. フィルタリング
1. セグメンテーション
1. 特徴量抽出
1. 特徴量選択・次元削減
1. 正規化・標準化

### 今回の手順
1. クレンジング
   - 欠損値の補完
1. 特徴量抽出
2. データ正規化・標準化


In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
from pathlib import Path
import json

In [ ]:
# データディレクトリの設定
DATA_DIR = Path("./data")

# 特徴量抽出時のウィンドウ幅
WINDOW_SIZE_SEC = 5

# 特徴量抽出時のウィンドウのステップ幅
WINDOW_STEP_SEC = 0.5


In [ ]:
# ラベル一覧の取得
# indexをIDとして使用する．LightGBM はラベル名が数値の連番である必要があるため
df_labels = pd.read_csv(DATA_DIR / "labels.csv", names=["label", "description"])
print(df_labels)


In [ ]:
# データ名の取得 (DATA_DIR内のフォルダ名をリストとして取得)
data_names = [f.name for f in DATA_DIR.iterdir() if f.is_dir()]
print(f"データ名: {data_names}")


In [ ]:
# データを読み取り，整える
data = {}

for name in data_names:
    # 加速度の読み込み
    df_acc = pd.read_csv(
        DATA_DIR / name / "Accelerometer.csv", names=["time", "x", "y", "z"], header=1
    )
    # 角速度の読み込み
    df_gyro = pd.read_csv(
        DATA_DIR / name / "Gyroscope.csv", names=["time", "x", "y", "z"], header=1
    )

    # ラベルの読み込み
    with open(DATA_DIR / name / "labeling.json") as f:
        content = json.load(f)
        tricks = content[0]["tricks"]

    sensing_start = min(tricks, key=lambda x: x["start"])["start"]
    sensing_end = max(tricks, key=lambda x: x["end"])["end"]

    # sensing_start ～ sensing_end のデータのみ抽出
    df_acc = df_acc[(df_acc["time"] >= sensing_start) & (df_acc["time"] <= sensing_end)]
    df_gyro = df_gyro[
        (df_gyro["time"] >= sensing_start) & (df_gyro["time"] <= sensing_end)
    ]
    # 時刻を0スタートに変換
    df_acc["time"] = df_acc["time"] - sensing_start
    df_gyro["time"] = df_gyro["time"] - sensing_start

    # df_acc, df_gyroにラベルを追加
    others_id = df_labels[df_labels["label"] == "others"].index[0]
    df_acc["label"] = others_id  # 初期値は others に設定
    df_gyro["label"] = others_id  # 初期値は others に設定

    for trick in tricks:
        label = trick["labels"]
        start_time = trick["start"]
        end_time = trick["end"]
        # 今回は同時に複数動作は行わない前提のため、最初のラベルのみ使用
        label = trick["labels"][0]
        # ラベル名からラベルIDを取得
        label_id = df_labels[df_labels["label"] == label].index[0]

        # df_accにラベルを追加
        df_acc.loc[
            (df_acc["time"] >= start_time) & (df_acc["time"] <= end_time), "label"
        ] = label_id

        # df_gyroにラベルを追加
        df_gyro.loc[
            (df_gyro["time"] >= start_time) & (df_gyro["time"] <= end_time), "label"
        ] = label_id

    data[name] = {"acc": df_acc, "gyro": df_gyro}

In [ ]:
def extract_features(df, window_size, window_step, columns=["x", "y", "z", "norm"]):
    """
    ウィンドウ幅 window_size、ステップ幅 window_step で特徴量を抽出する関数
    比較的わかりやすいが，計算量が多くなるため遅い(numpyを使うと高速化できる)

    特徴量:
    - ラベル（最頻値）
    - 各軸の最小値，最大値，平均，分散，標準偏差，中央値，各軸のゼロ交差率
    """
    features = pd.DataFrame()

    # ラベル（最頻値）
    features["label"] = (
        df["label"]
        .rolling(window=window_size)
        .apply(lambda x: x.mode()[0])
        .iloc[window_size - 1 :: window_step]
        .reset_index(drop=True)
    )

    # 数値列の統計量
    stats = ["min", "max", "mean", "var", "std", "median"]
    for stat in stats:
        rolled = getattr(df[columns].rolling(window=window_size), stat)()
        # 列名を x_min, y_min, z_min のように変更して結合
        rolled = rolled.iloc[window_size - 1 :: window_step].reset_index(drop=True)
        rolled.columns = [f"{col}_{stat}" for col in rolled.columns]
        features = pd.concat([features, rolled], axis=1)

    # ゼロ交差率
    zcr = (
        df[columns]
        .rolling(window=window_size)
        .apply(lambda x: ((x[:-1] * x[1:]) < 0).sum())
        .iloc[window_size - 1 :: window_step]
        .reset_index(drop=True)
    )
    # 列名を x_zcr, y_zcr, z_zcr に変更
    zcr.columns = [f"{col}_zcr" for col in zcr.columns]
    features = pd.concat([features, zcr], axis=1)

    return features


def extract_features_fast(
    df, window_size, window_step, columns=["x", "y", "z", "norm"]
):
    """
    extract_features関数の高速版(らしい)
    TODO: 実装
    """
    raise NotImplementedError("未実装")

In [ ]:
# データから特徴量を抽出し，保存
for name, dfs in data.items():
    df_acc = dfs["acc"]
    df_gyro = dfs["gyro"]

    # クレンジング(欠損値を線形補間で補完)
    df_acc = df_acc.interpolate()
    df_gyro = df_gyro.interpolate()

    # norm 列を追加
    df_acc["norm"] = np.sqrt(df_acc["x"] ** 2 + df_acc["y"] ** 2 + df_acc["z"] ** 2)
    df_gyro["norm"] = np.sqrt(df_gyro["x"] ** 2 + df_gyro["y"] ** 2 + df_gyro["z"] ** 2)

    # サンプリング周波数の計算
    fs_acc = 1 / (
        (df_acc["time"].iloc[-1] - df_acc["time"].iloc[0]) / (len(df_acc) - 1)
    )
    fs_gyro = 1 / (
        (df_gyro["time"].iloc[-1] - df_gyro["time"].iloc[0]) / (len(df_gyro) - 1)
    )

    # ウィンドウ幅 (サンプル数)
    acc_window_size = int(WINDOW_SIZE_SEC * fs_acc)
    gyro_window_size = int(WINDOW_SIZE_SEC * fs_gyro)

    # ウィンドウのステップ幅 (サンプル数)
    acc_window_step = int(WINDOW_STEP_SEC * fs_acc)
    gyro_window_step = int(WINDOW_STEP_SEC * fs_gyro)

    # 特徴量を抽出
    df_acc_features = extract_features(
        df_acc, acc_window_size, acc_window_step, columns=["x", "y", "z", "norm"]
    )
    df_gyro_features = extract_features(
        df_gyro, gyro_window_size, gyro_window_step, columns=["x", "y", "z", "norm"]
    )

    # 特徴量をCSVとして保存
    df_acc_features.to_csv(DATA_DIR / name / "acc_features.csv", index=False)
    df_gyro_features.to_csv(DATA_DIR / name / "gyro_features.csv", index=False)